In [162]:
import wandb
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
wandb.init(project="haselab-conpetition", name=f"experiment-{timestamp}")

In [163]:
import zipfile
from pathlib import Path
from PIL import Image
import subprocess

import torch
import torchvision
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.nn as nn

In [164]:
# データセットをダウンロード・展開するコード
# 学外にいる場合や，Google Colaboratory で実行する場合，このコードはうまく動かない．
# データセットをアクセスできる場所に置き，dataset_dir にデータセットのパスを指定する必要がある．

# 具体的手順は以下
# Google Colaboratory で実行する場合
# 1. Aの部分をコメントアウト
# 2. zipファイルをアップロードする．
# 3. Bの部分をアンコメント

# それ以外
# 1. competition_images/ をアクセスできる場所に置く
# 2. Aの部分をコメントアウト
# 3. Cをアンコメントし，competitions_images/ のパスとなる文字列を格納  Ex.) dataset_dir = "/home/haselab/Documents/tat/tmp/competition_images"



#      ----- A -----
#      # データセットの保存先を指定
#      save_dir = str(Path().resolve()) # 保存ディレクトリをノートブックと同じディレクトリに設定
#      dataset_dir = save_dir + "/competition_images/"
#      # データセットを保存
#      if Path(dataset_dir).exists():
#          print(f"Dataset directory already exists: {dataset_dir}")
#      else:
#          zip_path = Path(str(save_dir)) / "tmp.zip"
#          url = "http://10.0.87.42:8080/dataset" # こっちでもいける
#          subprocess.run(["wget", url, "-O", zip_path, "--no-proxy", "-q"], check=True) # .pyならこっち
#          # !wget -O {zip_path} {url} -q --no-proxy
#          with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#              zip_ref.extractall(save_dir)
#              if zip_path.exists():
#                  zip_path.unlink()
#          print(f"Downloaded to {save_dir} and extracted.")
#      ----- A -----



# ----- B -----
# zip_path = "/content/competition_images.zip" # zipファイルのパスを指定
# ext_dir = "/content/"  # 展開先を指定

# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(ext_dir)
# dataset_dir = ext_dir + "competition_images/" # データセットのパスを指定
# ----- B -----


# ----- C -----
dataset_dir = "./content/competition_images" # 要変更．データセットのパスを指定
# ----- C -----


In [165]:
# テストデータ読み込みのためのカスタムデータセットクラス
class TestDataset(Dataset):
    def __init__(self, path, transform=None, target_transform=None):
        self.img_paths = sorted([p for p in Path(path).iterdir()])
        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self, index):
        path = self.img_paths[index]
        data = Image.open(path).convert('RGB')

        if self.transform: # 画像の前処理
            data = self.transform(data)
        if self.target_transform:
            path = self.target_transform(path) # 画像のパスに前処理している。名前を変更？
        return data, str(path.name)

    def __len__(self):
        return len(self.img_paths)

In [166]:
# ハイパーパラメータを指定
epochs = 12
lr = 0.0001
batch_size = 256

wandb.config.update({
    "epochs": epochs,
    "learning_rate": lr,
    "batch_size": batch_size
})

In [167]:
from torch.utils.data import DataLoader, random_split

class CustomDataset(Dataset):
    def __init__(self, full_dataset, indices, transform=None):
        self.full_dataset = full_dataset
        self.indices = indices
        self.transform = transform

    def __getitem__(self, idx):
        index = self.indices[idx]
        image, label = self.full_dataset[index]

        if self.transform:
            image = self.transform(image)
        return image, label

    def __len__(self):
        return len(self.indices)

In [168]:
# モデルと訓練に必要なものを定義
num_classes = 4

train_tf = torchvision.transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.08, 1.0)), #80%~100%の範囲で切り取る
    transforms.RandomHorizontalFlip(), #?ランダムに反転->かえって精度が落ちた。
    transforms.RandomRotation(15), #?ランダムに回転
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1), #?色の変化
    
    transforms.ToTensor(),                                        #?追加で反転とか
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_tf = torchvision.transforms.Compose([

    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) #?平均と重みを事前訓練時の値に変更
])

test_tf = torchvision.transforms.Compose([

    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) #?平均と重みを事前訓練時の値に変更
])


#分割--------------------------------------------------------------------------------
temp_ds = ImageFolder(root=dataset_dir + "/train_val")
dataset_size = len(temp_ds)
val_split = 0.2
val_count = int(dataset_size * val_split)
train_count = dataset_size - val_count

generator = torch.Generator().manual_seed(89) #? 乱数シードを固定
train_subset_indices, val_subset_indices = random_split(range(dataset_size), [train_count, val_count], generator = generator)

train_ds = CustomDataset(temp_ds, train_subset_indices, transform=train_tf) #? train_tfを適用
val_ds = CustomDataset(temp_ds, val_subset_indices, transform=val_tf) #? val_tfを適用

val_dl = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=0)
#-------------------------------------------------------------------------------------

#train_ds = ImageFolder(root=dataset_dir + "/train_val", transform=train_tf)
test_ds = TestDataset(path=dataset_dir + "/test", transform=test_tf)


train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

# model = torchvision.models.resnet18(num_classes=num_classes) # 事前学習済みモデルを使用しない重みはランダム初期化

model = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.DEFAULT) # 事前学習済みモデルを使用する。ファインチューニング
num_ftrs = model.fc.in_features # 元の全結合層の入力特徴数を入手

#ドロップアウト層を追加
model.fc = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(num_ftrs, num_classes) # 全結合層を新しく定義
)
# model.fc = nn.Linear(num_ftrs, num_classes) # 全結合層を新しく定義



# for param in model.parameters():
#     param.requires_grad = False # まず全ての層の勾配計算をオフ
# for param in model.fc.parameters():
#     param.requires_grad = True # fc層のみ勾配計算をオン



criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=5e-4) #? 
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9) #? 学習率を0.9倍にするスケジューラ


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("using device : " + ( "GPU" if device.type == "cuda" else "CPU" )) #どっちつかってるか確認

using device : GPU


In [169]:
# 訓練フローを定義
def train_1epoch(model, train_dl, criterion, optimizer, scheduler=None, device="cpu"):
    total_loss = 0.0
    total_corr = 0

    model.train()                                       # 訓練モードに設定
    for inputs, labels in train_dl:                     # ミニバッチを取得してループ
        inputs = inputs.to(device)                      # GPU に転送
        labels = labels.to(device)                      # GPU に転送

        outputs = model(inputs)                         # 順伝播

        loss = criterion(outputs, labels)               # 損失計算
        preds = torch.argmax(outputs.detach(), dim=1)   # 予測値を取得 各データの予測の最大値のインデックスを収集する操作なので、別に追跡する必要はないってかしたら勾配計算が異常になるのでdetach()。
        corr = torch.sum(preds == labels.data).item()   # 正解数をカウント

        optimizer.zero_grad()                           # 勾配を初期化
        loss.backward()                                 # 誤差逆伝播
        optimizer.step()                                # 重み更新

        total_loss += loss.item() * len(inputs)         # 損失を累積
        total_corr += corr                              # 正解数を累積

    # if scheduler is not None:                           # 学習率スケジューラが指定されている場合
    #     scheduler.step()                                # 学習率を更新
    train_loss = total_loss / len(train_dl.dataset)     # 平均損失を計算
    train_acc = total_corr / len(train_dl.dataset)      # 平均精度を計算

    return train_loss, train_acc

In [170]:
def validate_1epoch(model, val_dl, criterion, device="cpu"):
    total_loss = 0.0
    total_corr = 0

    model.eval()                                        # 評価モードに設定
    with torch.no_grad():                               # 勾配計算を無効化
        for inputs, labels in val_dl:                   # ミニバッチを取得してループ
            inputs = inputs.to(device)                  # GPU に転送
            labels = labels.to(device)                  # GPU に転送

            outputs = model(inputs)                     # 順伝播

            loss = criterion(outputs, labels)           # 損失計算
            preds = torch.argmax(outputs, dim=1)        # 予測値を取得
            corr = torch.sum(preds == labels.data).item()    # 正解数をカウント

            total_loss += loss.item() * len(inputs)     # 損失を累積
            total_corr += corr                          # 正解数を累積

    val_loss = total_loss / len(val_dl.dataset)         # 平均損失を計算
    val_acc = total_corr / len(val_dl.dataset)          # 平均精度を計算

    return val_loss, val_acc

In [171]:
import time

# 推論フローを定義
def pred(model, test_dl, device, probs=False, categorize=False):
    total_outputs = []
    all_labels = []

    model.eval()
    with torch.no_grad():
        print("推論ループが開始された")
        loop_start_time = time.time() #ループ開始時刻


        for i, (inputs, labels) in enumerate(test_dl):
            print(f"Processing batch {i+1}/{len(test_dl)}...")
            batch_start_time = time.time() #バッチ開始時刻

            inputs = inputs.to(device)

            outputs = model(inputs)

            if probs:
                outputs = torch.softmax(outputs, dim=1)

            if categorize:
                outputs = torch.argmax(outputs, dim=1)

            total_outputs.append(outputs)
            all_labels.extend(labels)
            
            batch_end_time = time.time() #バッチ終了時刻
            if (i + 1) % 10 == 0: # 10バッチごとに進捗を表示
                print(f"Processed batch {i+1}/{len(test_dl)}, time per batch: {batch_end_time - batch_start_time:.4f}s")

        loop_end_time = time.time()
        print(f"Prediction loop finished in {loop_end_time - loop_start_time:.2f} seconds.")




    outputs = torch.cat(total_outputs, dim=0).cpu().tolist()
    return outputs, all_labels

In [172]:
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=2, verbose=True
)

c:\Users\furaf\anaconda3\envs\torch-env\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [173]:
# 訓練
model = model.to(device)
for epoch in range(epochs):
    train_loss, train_acc = train_1epoch(model, train_dl, criterion, optimizer, scheduler, device=device)
    print(f"epoch {epoch+1:>3}/{epochs:>3}: train_loss: {train_loss:.4f}, train_acc: {train_acc:.4f}")

    val_loss, val_acc = validate_1epoch(model, val_dl, criterion, device=device)
    print(f"epoch {epoch+1:>3}/{epochs:>3}: val_loss: {val_loss:.4f}, val_acc: {val_acc:.4f}")
    
    if scheduler is not None:                           # 学習率スケジューラが指定されている場合
        scheduler.step(val_loss)                                # 学習率を更新

    wandb.log({"val_accuracy": val_acc, "val_loss": val_loss}) # wandb にログを記録
    wandb.log({"train_accuracy": train_acc, "train_loss": train_loss}) # wandb にログを記録

wandb.save(f"model_epoch{epoch+1}.pth") # wandb にモデルを保存
wandb.finish() # wandb のセッションを終了

epoch   1/ 12: train_loss: 1.2650, train_acc: 0.4458
epoch   1/ 12: val_loss: 1.7727, val_acc: 0.2625
epoch   2/ 12: train_loss: 0.7178, train_acc: 0.7260
epoch   2/ 12: val_loss: 1.1925, val_acc: 0.4167
epoch   3/ 12: train_loss: 0.5939, train_acc: 0.7927
epoch   3/ 12: val_loss: 0.9338, val_acc: 0.5833
epoch   4/ 12: train_loss: 0.4865, train_acc: 0.8292
epoch   4/ 12: val_loss: 0.8891, val_acc: 0.6417
epoch   5/ 12: train_loss: 0.4361, train_acc: 0.8500
epoch   5/ 12: val_loss: 0.8586, val_acc: 0.6750
epoch   6/ 12: train_loss: 0.4377, train_acc: 0.8448
epoch   6/ 12: val_loss: 0.9167, val_acc: 0.6708
epoch   7/ 12: train_loss: 0.4077, train_acc: 0.8448
epoch   7/ 12: val_loss: 0.9482, val_acc: 0.6667
epoch   8/ 12: train_loss: 0.3904, train_acc: 0.8698
epoch   8/ 12: val_loss: 1.0637, val_acc: 0.6542
epoch   9/ 12: train_loss: 0.3743, train_acc: 0.8646
epoch   9/ 12: val_loss: 1.1058, val_acc: 0.6583
epoch  10/ 12: train_loss: 0.3675, train_acc: 0.8646
epoch  10/ 12: val_loss: 1.17

train_accuracy,▁▆▇▇▇▇▇█████
train_loss,█▄▃▂▂▂▂▂▁▁▁▁
val_accuracy,▁▄▆▇████████
val_loss,█▄▂▁▁▁▂▃▃▃▃▃
train_accuracy,0.88125
train_loss,0.30757
val_accuracy,0.675
val_loss,1.14447


In [174]:
# 推論・結果をCSV形式で保存

csv_name = f"result/predictions{timestamp}.csv"

outputs, all_labels = pred(model, test_dl, device, categorize=True)
lines = [f"{l},{o}" for o, l in zip(outputs, all_labels)]
csv_text = "\n".join(lines)

with open(csv_name, "w", encoding="utf-8") as f:
    f.write(csv_text)


推論ループが開始された
Processing batch 1/9...
Processing batch 2/9...
Processing batch 3/9...
Processing batch 4/9...
Processing batch 5/9...
Processing batch 6/9...
Processing batch 7/9...
Processing batch 8/9...
Processing batch 9/9...
Prediction loop finished in 0.85 seconds.


変更点
モデルのresnet18を使用
事前に訓練された重みを利用＋最後の結合層をカスタマイズ（ファインチューニング）
ハイパラは以下のように
epochs = 10
lr = 0.001
batch_size = 256
オプティマイザをAdamWに変更
スケジューラの追加